# PipInstall

#### sk10841
#### ss17526
#### ap8589

## "Music-Based User Matchmaking System: Connecting People Through Shared Music Preferences"

### Connecting to spark

In [1]:
import os
import pyspark

conf = pyspark.SparkConf()
conf = conf.setAppName("Proj")
# conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040') ## to setup SPARK UI
# conf = conf.set('spark.jars', os.environ['GRAPHFRAMES_PATH']) ## graphframes in spark configuration
conf.set("spark.driver.bindAddress", "127.0.0.1")
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','8g')
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.mapKeyDedupPolicy", "LAST_WIN")

sc = pyspark.SparkContext(conf=conf)
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 13:17:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=Proj>

In [2]:
spark = pyspark.SQLContext.getOrCreate(sc)
spark

/Users/sudhanshusali/miniconda3/lib/python3.12/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
!pip install OpenAI

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)


In [79]:
#Loading the necessary libraries

from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np
from pyspark.sql.types import FloatType
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from openai import OpenAI

In [5]:
os.environ["OPENAI_API_KEY"] = "key"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

### Loading the dataset

In [6]:
#Load the datasets
tracks_df = spark.read.csv("data/tracks.csv", header=True, inferSchema=True)
cust_df = spark.read.csv("data/cust.csv", header=True, inferSchema=True)
music_df = spark.read.csv("data/music.csv", header=True, inferSchema=True)

In [7]:
sc._jsc.sc().getExecutorMemoryStatus().size()

1

In [8]:
tracks_df.limit(5)

EventID,CustID,TrackId,DateTime,Mobile,ZipCode
0,48,453,10/23/14 3:26,0,72132
1,1081,19,10/15/14 18:32,1,17307
2,532,36,12/10/14 15:33,1,66216
3,2641,822,10/20/14 2:24,1,36690
4,2251,338,11/18/14 7:16,1,61377


In [9]:
tracks_df.dtypes

[('EventID', 'int'),
 ('CustID', 'int'),
 ('TrackId', 'int'),
 ('DateTime', 'string'),
 ('Mobile', 'int'),
 ('ZipCode', 'int')]

### Exploratory Data Analysis

In [10]:
# Ensuring the data is clean
null_counts = tracks_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df.columns])
null_counts.show()

+-------+------+-------+--------+------+-------+
|EventID|CustID|TrackId|DateTime|Mobile|ZipCode|
+-------+------+-------+--------+------+-------+
|      0|     0|      0|       0|     0|      0|
+-------+------+-------+--------+------+-------+



In [11]:
#Displaying shape
(tracks_df.count(), len(tracks_df.columns))

(1000000, 6)

In [12]:
cust_df.limit(5)

CustID,Name,Gender,Address,zip,SignDate,Status,Level,Campaign,LinkedWithApps
0,Gregory Koval,0,13004 Easy Cider ...,72132,06/04/2013,1,1,1,0
1,Robert Gordon,0,10497 Thunder Hic...,17307,07/27/2013,1,1,1,0
2,Paula Peltier,0,10084 Easy Gate Bend,66216,01/13/2013,1,0,4,1
3,Francine Gray,0,54845 Bent Pony H...,36690,07/11/2013,1,1,1,1
4,David Garcia,0,8551 Tawny Fox Villa,61377,09/09/2012,1,0,1,1


In [13]:
cust_df.dtypes

[('CustID', 'int'),
 ('Name', 'string'),
 ('Gender', 'int'),
 ('Address', 'string'),
 ('zip', 'int'),
 ('SignDate', 'string'),
 ('Status', 'int'),
 ('Level', 'int'),
 ('Campaign', 'int'),
 ('LinkedWithApps', 'int')]

In [14]:
null_counts = cust_df.select([count(when(col(c).isNull(), c)).alias(c) for c in cust_df.columns])
null_counts.show()

+------+----+------+-------+---+--------+------+-----+--------+--------------+
|CustID|Name|Gender|Address|zip|SignDate|Status|Level|Campaign|LinkedWithApps|
+------+----+------+-------+---+--------+------+-----+--------+--------------+
|     0|   0|     0|      0|  0|       0|     0|    0|       0|             0|
+------+----+------+-------+---+--------+------+-----+--------+--------------+



In [15]:
#Displaying shape
(cust_df.count(), len(cust_df.columns))

(5000, 10)

In [16]:
music_df.limit(5)

TrackId,Title,Artist,Length
0,Caught Up In You,.38 Special,200
1,Fantasy Girl,.38 Special,219
2,Hold On Loosely,.38 Special,253
3,Hold On Loosely ...,.38 Special,154
4,Art For Arts Sake,10cc,341


In [17]:
music_df.dtypes

[('TrackId', 'int'),
 ('Title', 'string'),
 ('Artist', 'string'),
 ('Length', 'int')]

In [18]:
null_counts = music_df.select([count(when(col(c).isNull(), c)).alias(c) for c in music_df.columns])
null_counts.show()

+-------+-----+------+------+
|TrackId|Title|Artist|Length|
+-------+-----+------+------+
|      0|    0|     0|    15|
+-------+-----+------+------+



In [19]:
#Displaying shape
(music_df.count(), len(music_df.columns))

(1750, 4)

### Data Preprocessing

In [20]:
#Making column names consistent
tracks_df = tracks_df.withColumnRenamed("ZipCode", "ListeningZIP").withColumnRenamed("CustID", "CustomerID")
cust_df = cust_df.withColumnRenamed("zip", "ZIP").withColumnRenamed("CustID", "CustomerID")

In [21]:
#converting to date dtype
cust_df = cust_df.withColumn("SignDate", to_date("SignDate", "MM/dd/yyyy"))

In [22]:
null_counts = cust_df.select([count(when(col(c).isNull(), c)).alias(c) for c in cust_df.columns])
null_counts.show()

+----------+----+------+-------+---+--------+------+-----+--------+--------------+
|CustomerID|Name|Gender|Address|ZIP|SignDate|Status|Level|Campaign|LinkedWithApps|
+----------+----+------+-------+---+--------+------+-----+--------+--------------+
|         0|   0|     0|      0|  0|       0|     0|    0|       0|             0|
+----------+----+------+-------+---+--------+------+-----+--------+--------------+



In [23]:
cust_df.limit(5)

CustomerID,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps
0,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0
1,Robert Gordon,0,10497 Thunder Hic...,17307,2013-07-27,1,1,1,0
2,Paula Peltier,0,10084 Easy Gate Bend,66216,2013-01-13,1,0,4,1
3,Francine Gray,0,54845 Bent Pony H...,36690,2013-07-11,1,1,1,1
4,David Garcia,0,8551 Tawny Fox Villa,61377,2012-09-09,1,0,1,1


In [24]:
tracks_df = tracks_df.withColumn("DateTime", to_timestamp("DateTime", "M/d/yy H:mm"))

In [25]:
tracks_df.limit(5)

EventID,CustomerID,TrackId,DateTime,Mobile,ListeningZIP
0,48,453,2014-10-23 03:26:00,0,72132
1,1081,19,2014-10-15 18:32:00,1,17307
2,532,36,2014-12-10 15:33:00,1,66216
3,2641,822,2014-10-20 02:24:00,1,36690
4,2251,338,2014-11-18 07:16:00,1,61377


In [26]:
null_counts = tracks_df.select([count(when(col(c).isNull(), c)).alias(c) for c in tracks_df.columns])
null_counts.show()

+-------+----------+-------+--------+------+------------+
|EventID|CustomerID|TrackId|DateTime|Mobile|ListeningZIP|
+-------+----------+-------+--------+------+------------+
|      0|         0|      0|       0|     0|           0|
+-------+----------+-------+--------+------+------------+



In [27]:
music_df.filter(col("Length").isNull()).show()

+-------+--------------------+--------------------+------+
|TrackId|               Title|              Artist|Length|
+-------+--------------------+--------------------+------+
|    326|   Helplessly Hoping|          Crosby,195|  NULL|
|    327|      Long Time Gone|    David Crosby,229|  NULL|
|    328|      Southern Cross|          Crosby,204|  NULL|
|    330|            Carry On|          Crosby,271|  NULL|
|    331|                Ohio|          Crosby,180|  NULL|
|    332|           Woodstock|          Crosby,147|  NULL|
|    480|  From the Beginning|         Emerson,264|  NULL|
|    481|             Hoedown|         Emerson,205|  NULL|
|    483|           Lucky Man|         Emerson,359|  NULL|
|    484|Still You Turn Me On|         Emerson,228|  NULL|
|    602|    Marakesh Express|Gold Star Ballroo...|  NULL|
|    830|     Just Dropped In|    Kenny Rogers,139|  NULL|
|    927|        Conquistador|London Symphony O...|  NULL|
|   1202|         In The Mood|           Plant,271|  NUL

In [28]:
#Dropping the na values since they are less than 5% of the total
music_df = music_df.na.drop(subset=["Length"])
music_df.show(5)

+-------+--------------------+-----------+------+
|TrackId|               Title|     Artist|Length|
+-------+--------------------+-----------+------+
|      0|    Caught Up In You|.38 Special|   200|
|      1|        Fantasy Girl|.38 Special|   219|
|      2|     Hold On Loosely|.38 Special|   253|
|      3|Hold On Loosely  ...|.38 Special|   154|
|      4|   Art For Arts Sake|       10cc|   341|
+-------+--------------------+-----------+------+
only showing top 5 rows



In [29]:
#Displaying shape
(music_df.count(), len(music_df.columns))

(1735, 4)

In [30]:
# Join datasets
songs = tracks_df.join(music_df, on="TrackId", how="inner")

In [31]:
songs.limit(10)

TrackId,EventID,CustomerID,DateTime,Mobile,ListeningZIP,Title,Artist,Length
453,0,48,2014-10-23 03:26:00,0,72132,Strange Magic,Electric Light Or...,170
19,1,1081,2014-10-15 18:32:00,1,17307,Money Talks,AC/DC,323
36,2,532,2014-12-10 15:33:00,1,66216,Big Ten Inch Record,Aerosmith,204
822,3,2641,2014-10-20 02:24:00,1,36690,The Ripper,Judas Priest,122
338,4,2251,2014-11-18 07:16:00,1,61377,Welcome To The Bo...,David & David,269
6,5,1811,2014-11-18 02:00:00,1,20115,Loser,3 Doors Down,127
24,6,3644,2014-12-12 15:24:00,1,15330,Shot Down In Flames,AC/DC,326
726,7,250,2014-10-07 09:48:00,0,33570,Right Here Right Now,Jesus Jones,232
442,8,1782,2014-12-30 15:27:00,1,41240,Baby Hold On,Eddie Money,298
775,9,2932,2014-11-12 07:45:00,0,63565,On the Dark Side,John Cafferty,213


In [32]:
#Combined all three datasets
joined_df = songs.join(cust_df, on="CustomerID", how="inner")

In [33]:
joined_df.limit(10)

CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps
48,453,0,2014-10-23 03:26:00,0,72132,Strange Magic,Electric Light Or...,170,Lucas Pizano,0,2723 Stony Beaver...,99256,2012-11-02,1,0,3,1
1081,19,1,2014-10-15 18:32:00,1,17307,Money Talks,AC/DC,323,Kenneth Rodgers,0,74413 Heather Elm...,30301,2013-05-14,0,1,1,1
532,36,2,2014-12-10 15:33:00,1,66216,Big Ten Inch Record,Aerosmith,204,Carlos Kirk,0,14 Hidden Bear Ci...,90745,2013-07-14,0,2,1,1
2641,822,3,2014-10-20 02:24:00,1,36690,The Ripper,Judas Priest,122,Charlene Boyd,0,5967 Stony Branch...,4645,2013-03-26,1,1,1,0
2251,338,4,2014-11-18 07:16:00,1,61377,Welcome To The Bo...,David & David,269,Mary Decker,0,16355 Pretty Pand...,40580,2013-06-23,1,1,0,1
1811,6,5,2014-11-18 02:00:00,1,20115,Loser,3 Doors Down,127,Joshua Reyna,0,987 Cotton Maple ...,24574,2013-01-14,1,1,1,1
3644,24,6,2014-12-12 15:24:00,1,15330,Shot Down In Flames,AC/DC,326,Cynthia Moses,1,86610 Misty Necta...,23857,2013-06-25,1,0,1,0
250,726,7,2014-10-07 09:48:00,0,33570,Right Here Right Now,Jesus Jones,232,Nell Rosenblum,1,18820 Birch Mill ...,72443,2012-12-05,1,0,1,1
1782,442,8,2014-12-30 15:27:00,1,41240,Baby Hold On,Eddie Money,298,Jason Witzel,0,76 Jagged Barn Turn,28086,2013-07-19,1,2,3,1
2932,775,9,2014-11-12 07:45:00,0,63565,On the Dark Side,John Cafferty,213,Darlene Rahn,0,4 Tawny Beaver Ledge,7737,2013-05-11,1,0,1,0


In [34]:
null_counts = joined_df.select([count(when(col(c).isNull(), c)).alias(c) for c in joined_df.columns])
null_counts.show()

+----------+-------+-------+--------+------+------------+-----+------+------+----+------+-------+---+--------+------+-----+--------+--------------+
|CustomerID|TrackId|EventID|DateTime|Mobile|ListeningZIP|Title|Artist|Length|Name|Gender|Address|ZIP|SignDate|Status|Level|Campaign|LinkedWithApps|
+----------+-------+-------+--------+------+------------+-----+------+------+----+------+-------+---+--------+------+-----+--------+--------------+
|         0|      0|      0|       0|     0|           0|    0|     0|     0|   0|     0|      0|  0|       0|     0|    0|       0|             0|
+----------+-------+-------+--------+------+------------+-----+------+------+----+------+-------+---+--------+------+-----+--------+--------------+



In [35]:
#Displaying shape
(joined_df.count(), len(joined_df.columns))

(991755, 18)

### Feature Engineering

In [36]:
# Time-based features
joined_df = joined_df.withColumn("DateTime", to_timestamp("DateTime"))
joined_df = joined_df.withColumn("DayOfWeek", dayofweek("DateTime"))
joined_df = joined_df.withColumn("HourOfDay", hour("DateTime"))
joined_df = joined_df.withColumn("IsWeekend", when((col("DayOfWeek") == 1) | (col("DayOfWeek") == 7), 1).otherwise(0))

In [37]:
joined_df.limit(10)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=44453Kb max_used=44453Kb free=86618Kb
 bounds [0x0000000101f2c000, 0x0000000104adc000, 0x0000000109f2c000]
 total_blobs=15043 nmethods=14083 adapters=871
 compilation: disabled (not enough contiguous free space left)


CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps,DayOfWeek,HourOfDay,IsWeekend
48,453,0,2014-10-23 03:26:00,0,72132,Strange Magic,Electric Light Or...,170,Lucas Pizano,0,2723 Stony Beaver...,99256,2012-11-02,1,0,3,1,5,3,0
1081,19,1,2014-10-15 18:32:00,1,17307,Money Talks,AC/DC,323,Kenneth Rodgers,0,74413 Heather Elm...,30301,2013-05-14,0,1,1,1,4,18,0
532,36,2,2014-12-10 15:33:00,1,66216,Big Ten Inch Record,Aerosmith,204,Carlos Kirk,0,14 Hidden Bear Ci...,90745,2013-07-14,0,2,1,1,4,15,0
2641,822,3,2014-10-20 02:24:00,1,36690,The Ripper,Judas Priest,122,Charlene Boyd,0,5967 Stony Branch...,4645,2013-03-26,1,1,1,0,2,2,0
2251,338,4,2014-11-18 07:16:00,1,61377,Welcome To The Bo...,David & David,269,Mary Decker,0,16355 Pretty Pand...,40580,2013-06-23,1,1,0,1,3,7,0
1811,6,5,2014-11-18 02:00:00,1,20115,Loser,3 Doors Down,127,Joshua Reyna,0,987 Cotton Maple ...,24574,2013-01-14,1,1,1,1,3,2,0
3644,24,6,2014-12-12 15:24:00,1,15330,Shot Down In Flames,AC/DC,326,Cynthia Moses,1,86610 Misty Necta...,23857,2013-06-25,1,0,1,0,6,15,0
250,726,7,2014-10-07 09:48:00,0,33570,Right Here Right Now,Jesus Jones,232,Nell Rosenblum,1,18820 Birch Mill ...,72443,2012-12-05,1,0,1,1,3,9,0
1782,442,8,2014-12-30 15:27:00,1,41240,Baby Hold On,Eddie Money,298,Jason Witzel,0,76 Jagged Barn Turn,28086,2013-07-19,1,2,3,1,3,15,0
2932,775,9,2014-11-12 07:45:00,0,63565,On the Dark Side,John Cafferty,213,Darlene Rahn,0,4 Tawny Beaver Ledge,7737,2013-05-11,1,0,1,0,4,7,0


In [38]:
# joined_df.limit(10).toPandas()

In [39]:
# User listening frequency
window_spec = Window.partitionBy("CustomerID")
joined_df = joined_df.withColumn("UserListeningCount", count("EventID").over(window_spec))

In [40]:
joined_df.limit(10)
# joined_df.limit(10).toPandas()

CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps,DayOfWeek,HourOfDay,IsWeekend,UserListeningCount
26,146,388,2014-11-09 16:02:00,0,83274,Baby Come Back,Billy Rankin,352,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,1,16,1,1356
26,210,2454,2014-11-04 06:02:00,1,77375,Living On A Prayer,Bon Jovi,240,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,6,0,1356
26,8,2475,2014-12-26 20:39:00,0,33307,Back In Black,AC/DC,304,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,20,0,1356
26,195,2613,2014-12-20 22:16:00,0,23293,Katmandu,Bob Seger,173,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,7,22,1,1356
26,141,2626,2014-12-12 12:02:00,0,54175,SCENES FROM AN IT...,BILLY JOEL,269,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,12,0,1356
26,69,2997,2014-12-23 10:19:00,0,78344,House Of Fire,Alice Cooper,224,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,10,0,1356
26,1351,4296,2014-10-06 05:26:00,0,51354,Abracadabra,Steve Miller,323,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,2,5,0,1356
26,5,4600,2014-12-07 20:48:00,0,72165,Kryptonite,3 Doors Down,221,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,1,20,1,1356
26,787,5437,2014-10-27 15:00:00,1,63563,Cherry Bomb,John Mellencamp,163,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,2,15,0,1356
26,60,7481,2014-10-28 09:53:00,0,49834,You See Me Crying,Aerosmith,163,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,9,0,1356


In [41]:
# Track popularity
track_window = Window.partitionBy("TrackId")
joined_df = joined_df.withColumn("TrackPopularity", count("EventID").over(track_window))

In [42]:
# User-Track interaction features
user_track_window = Window.partitionBy("CustomerID", "TrackId")
joined_df = joined_df.withColumn("UserTrackInteractionCount", count("EventID").over(user_track_window))

In [43]:
joined_df.limit(10)
# joined_df.limit(10).toPandas()

CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps,DayOfWeek,HourOfDay,IsWeekend,UserListeningCount,TrackPopularity,UserTrackInteractionCount
0,26,114104,2014-12-21 10:50:00,1,74844,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,1,10,1,7034,2434,17
0,26,239562,2014-12-18 02:31:00,1,5670,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,5,2,0,7034,2434,17
0,26,258635,2014-11-20 22:25:00,0,25276,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,5,22,0,7034,2434,17
0,26,325147,2014-10-23 06:06:00,1,20408,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,5,6,0,7034,2434,17
0,26,353817,2014-10-10 19:25:00,1,98009,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,6,19,0,7034,2434,17
0,26,363754,2014-12-03 01:17:00,0,27536,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,4,1,0,7034,2434,17
0,26,417707,2014-11-19 23:26:00,1,34740,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,4,23,0,7034,2434,17
0,26,491109,2014-10-12 00:48:00,1,64055,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,1,0,1,7034,2434,17
0,26,527388,2014-12-01 07:00:00,0,19895,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,2,7,0,7034,2434,17
0,26,552844,2014-12-08 05:55:00,1,18602,Thunderstruck,AC/DC,327,Gregory Koval,0,13004 Easy Cider ...,72132,2013-06-04,1,1,1,0,2,5,0,7034,2434,17


In [44]:
# Mobile usage ratio
joined_df = joined_df.withColumn("MobileUsageRatio", 
                                 sum("Mobile").over(window_spec) / count("EventID").over(window_spec))

In [45]:
# User tenure
joined_df = joined_df.withColumn("SignDate", to_date("SignDate"))
joined_df = joined_df.withColumn("UserTenure", datediff(current_date(), "SignDate"))

In [46]:
joined_df.limit(10)
# joined_df.limit(10).toPandas()

24/12/11 13:19:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps,DayOfWeek,HourOfDay,IsWeekend,UserListeningCount,TrackPopularity,UserTrackInteractionCount,MobileUsageRatio,UserTenure
26,0,67019,2014-12-16 01:21:00,1,23825,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,1,0,1356,12113,12,0.5154867256637168,4275
26,0,134577,2014-11-18 01:34:00,1,12944,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,1,0,1356,12113,12,0.5154867256637168,4275
26,0,160290,2014-10-04 06:51:00,1,27114,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,7,6,1,1356,12113,12,0.5154867256637168,4275
26,0,250326,2014-11-21 22:12:00,1,14170,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,22,0,1356,12113,12,0.5154867256637168,4275
26,0,314981,2014-10-17 23:04:00,1,3605,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,23,0,1356,12113,12,0.5154867256637168,4275
26,0,349377,2014-12-19 05:57:00,0,71447,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,5,0,1356,12113,12,0.5154867256637168,4275
26,0,424704,2014-10-30 19:09:00,0,30094,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,5,19,0,1356,12113,12,0.5154867256637168,4275
26,0,425799,2014-12-19 18:22:00,0,59729,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,18,0,1356,12113,12,0.5154867256637168,4275
26,0,512258,2014-10-31 21:36:00,0,17229,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,21,0,1356,12113,12,0.5154867256637168,4275
26,0,752288,2014-11-26 13:53:00,0,30313,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,4,13,0,1356,12113,12,0.5154867256637168,4275


In [47]:
# Genre features
artist_indexer = StringIndexer(inputCol="Artist", outputCol="ArtistIndex")
artist_encoder = OneHotEncoder(inputCol="ArtistIndex", outputCol="ArtistVector")

In [48]:
# Normalize continuous features
joined_df = joined_df.withColumn("NormalizedLength", col("Length") / 1000)  # Assuming Length is in seconds

In [49]:
joined_df.limit(10)
# joined_df.limit(10).toPandas()

CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps,DayOfWeek,HourOfDay,IsWeekend,UserListeningCount,TrackPopularity,UserTrackInteractionCount,MobileUsageRatio,UserTenure,NormalizedLength
26,0,67019,2014-12-16 01:21:00,1,23825,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,1,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,134577,2014-11-18 01:34:00,1,12944,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,1,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,160290,2014-10-04 06:51:00,1,27114,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,7,6,1,1356,12113,12,0.5154867256637168,4275,0.2
26,0,250326,2014-11-21 22:12:00,1,14170,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,22,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,314981,2014-10-17 23:04:00,1,3605,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,23,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,349377,2014-12-19 05:57:00,0,71447,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,5,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,424704,2014-10-30 19:09:00,0,30094,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,5,19,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,425799,2014-12-19 18:22:00,0,59729,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,18,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,512258,2014-10-31 21:36:00,0,17229,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,21,0,1356,12113,12,0.5154867256637168,4275,0.2
26,0,752288,2014-11-26 13:53:00,0,30313,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,4,13,0,1356,12113,12,0.5154867256637168,4275,0.2


In [50]:
# User listening diversity
# Create the user listening diversity feature
joined_df = joined_df.withColumn(
    "UserTrackDiversity",
    size(collect_set("TrackId").over(window_spec)) / count("EventID").over(window_spec)
)

In [51]:
# Recency feature
joined_df = joined_df.withColumn("LastListenDate", max("DateTime").over(window_spec))
joined_df = joined_df.withColumn("DaysSinceLastListen", datediff(current_date(), "LastListenDate"))

In [52]:
joined_df.limit(10)
# joined_df.limit(10).toPandas()

CustomerID,TrackId,EventID,DateTime,Mobile,ListeningZIP,Title,Artist,Length,Name,Gender,Address,ZIP,SignDate,Status,Level,Campaign,LinkedWithApps,DayOfWeek,HourOfDay,IsWeekend,UserListeningCount,TrackPopularity,UserTrackInteractionCount,MobileUsageRatio,UserTenure,NormalizedLength,UserTrackDiversity,LastListenDate,DaysSinceLastListen
26,0,67019,2014-12-16 01:21:00,1,23825,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,1,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,134577,2014-11-18 01:34:00,1,12944,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,3,1,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,160290,2014-10-04 06:51:00,1,27114,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,7,6,1,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,250326,2014-11-21 22:12:00,1,14170,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,22,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,314981,2014-10-17 23:04:00,1,3605,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,23,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,349377,2014-12-19 05:57:00,0,71447,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,5,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,424704,2014-10-30 19:09:00,0,30094,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,5,19,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,425799,2014-12-19 18:22:00,0,59729,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,18,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,512258,2014-10-31 21:36:00,0,17229,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,6,21,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634
26,0,752288,2014-11-26 13:53:00,0,30313,Caught Up In You,.38 Special,200,Pauline Mead,1,47575 Heather App...,17509,2013-03-29,1,0,4,1,4,13,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,2014-12-30 23:44:00,3634


In [53]:
# Apply the transformations
pipeline = Pipeline(stages=[artist_indexer, artist_encoder])
transformed_df = pipeline.fit(joined_df).transform(joined_df)

In [54]:
# Select relevant columns for the final feature set
feature_cols = ["CustomerID", "TrackId", "DayOfWeek", "HourOfDay", "IsWeekend", "UserListeningCount", 
                "TrackPopularity", "UserTrackInteractionCount", "MobileUsageRatio", "UserTenure", 
                "NormalizedLength", "UserTrackDiversity", "DaysSinceLastListen", "ArtistVector"]

In [55]:
final_df = transformed_df.select(feature_cols)

In [56]:
# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=[col for col in feature_cols if col not in ["CustomerID", "TrackId"]], 
                            outputCol="features")
final_features_df = assembler.transform(final_df)

### Final Dataset with all the required features

In [57]:
# Show the resulting dataframe
final_features_df.limit(10)
# final_features_df.toPandas()

CustomerID,TrackId,DayOfWeek,HourOfDay,IsWeekend,UserListeningCount,TrackPopularity,UserTrackInteractionCount,MobileUsageRatio,UserTenure,NormalizedLength,UserTrackDiversity,DaysSinceLastListen,ArtistVector,features
26,0,3,1,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,3,1,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,7,6,1,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,2,3,4,5..."
26,0,6,22,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,6,23,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,6,5,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,5,19,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,6,18,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,6,21,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."
26,0,4,13,0,1356,12113,12,0.5154867256637168,4275,0.2,0.6231563421828908,3634,"(507,[2],[1.0])","(518,[0,1,3,4,5,6..."


In [58]:
# # Save the feature engineered dataset
# final_features_df.write.parquet("music_recommendation_features.parquet")

## Implementing the ALS model to match users, find similar tracks and recommend new tracks
Each user (CustomerId) listens to a track (TrackId) some n number of times (UserTrackInteractionCount). Intuition is that the more a user listens to a track, the higher the likelihood of them rating it highly i.e., liking the song. We can now use collaborative filtering to find user similarity scores.

In [60]:
# Cosine similarity UDF
def cosine_similarity(v1, v2):
    v1_array = np.array(v1)
    v2_array = np.array(v2)
    return float(np.dot(v1_array, v2_array) / (np.linalg.norm(v1_array) * np.linalg.norm(v2_array)))

cosine_similarity_udf = udf(cosine_similarity, FloatType())

def get_recommendations(spark, joined_df, target_user_id, num_recommendations=10, num_similar_users=5, model=None):
    # Prepare data for ALS
    als_data = joined_df.select(
        col("CustomerID").alias("user"),
        col("TrackId").alias("item"),
        col("UserTrackInteractionCount").alias("rating")
    )

    # Split data into training and test sets
    (training, test) = als_data.randomSplit([0.8, 0.2])

    # Train a default ALS model
    if model is None:
        als = ALS(
            rank=10,
            maxIter=5,
            regParam=0.01,
            alpha = 0.1,
            userCol="user",
            itemCol="item",
            ratingCol="rating",
            coldStartStrategy="drop"
        )
        model = als.fit(training)

    # Make predictions on the test set
    predictions = model.transform(test)

    # Evaluate the model using RMSE and MAE
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)

    evaluator = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
    mae = evaluator.evaluate(predictions)

    # Get user factors for similarity calculation
    user_factors = model.userFactors

    # Get target user's features
    target_user_factor = user_factors.filter(col("id") == target_user_id).select("features").first()[0]

    if isinstance(target_user_factor, list):
        target_user_factor_list = target_user_factor
    else:
        target_user_factor_list = target_user_factor.toArray().tolist()

    similarity_df = user_factors.filter(col("id") != target_user_id).select(
        col("id").alias("similar_user"),
        cosine_similarity_udf(col("features"), lit(target_user_factor_list)).alias("similarity")
    )

    # Get top 5 similar users
    top_similar_users = similarity_df.orderBy(col("similarity").desc()).limit(num_similar_users)

    # Generate recommendations for the target user
    user_recs = model.recommendForUserSubset(
        spark.createDataFrame([(target_user_id,)], ["user"]), num_recommendations
    )

    # Extract recommended track IDs and scores
    recommended_tracks = user_recs.select(expr("explode(recommendations)")).select(
        col("col.item").alias("TrackId"),
    )

    # Get details of recommended tracks
    recommended_tracks = recommended_tracks.join(
        joined_df.select("TrackId", "Title", "Artist").distinct(),
        "TrackId"
    )

    return top_similar_users, recommended_tracks, rmse, mae

In [61]:
target_user_id = 500

# Recommendation using base model
similar_users, recommended_tracks, rmse, mae = get_recommendations(
    spark,
    joined_df,
    target_user_id,
    num_recommendations=10,
    num_similar_users=5
)

print(f"Model Evaluation:")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

print("\nTop 5 Similar Users:")
similar_users.show()

print("\nRecommended Tracks:")
recommended_tracks.show()

24/12/11 13:22:43 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/11 13:22:43 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Model Evaluation:
Root Mean Squared Error (RMSE): 0.5734261096741297
Mean Absolute Error (MAE): 0.34952338533832056

Top 5 Similar Users:


+------------+----------+
|similar_user|similarity|
+------------+----------+
|        3155| 0.9878468|
|        3952| 0.9854446|
|         961|0.98277175|
|         316|0.98224396|
|        1074| 0.9819884|
+------------+----------+


Recommended Tracks:


+-------+-----------------+------------+
|TrackId|            Title|      Artist|
+-------+-----------------+------------+
|      5|       Kryptonite|3 Doors Down|
|     17|Let There Be Rock|       AC/DC|
|      8|    Back In Black|       AC/DC|
|     12|   Hard As A Rock|       AC/DC|
|      4|Art For Arts Sake|        10cc|
|     21|    Night Prowler|       AC/DC|
|      7|       Take On Me|        a-ha|
|     20|       Moneytalks|       AC/DC|
|      2|  Hold On Loosely| .38 Special|
|      6|            Loser|3 Doors Down|
+-------+-----------------+------------+



## Hyperparameter Tuning

In [62]:
def tune_als_with_tvs(training_data):
    # Define ALS model
    als = ALS(
        userCol="user",
        itemCol="item",
        ratingCol="rating",
        coldStartStrategy="drop"
    )
    
    # Define parameter grid
    param_grid = ParamGridBuilder().addGrid(als.rank, [10, 20, 30, 40]).addGrid(als.regParam, [0.001, 0.01, 0.05, 0.1]).addGrid(als.maxIter, [5, 10, 20, 50]).addGrid(als.alpha, [0.1, 1.0, 10.0]).build()
    
    # Define evaluator
    evaluator = RegressionEvaluator(
        metricName="rmse", 
        labelCol="rating", 
        predictionCol="prediction"
    )
    
    # Define TrainValidationSplit
    tvs = TrainValidationSplit(
        estimator=als,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        trainRatio=0.8
    )
    
    # Fit the model using TrainValidationSplit
    model = tvs.fit(training_data)
    
    # Get the best model and parameters
    best_model = model.bestModel
    best_params = {
        "rank": best_model.rank,
        "regParam": best_model._java_obj.parent().getRegParam(),
        "maxIter": best_model._java_obj.parent().getMaxIter()
    }
    
    return best_model, best_params



In [74]:
# Prepare data for ALS
als_data = joined_df.select(
    col("CustomerID").alias("user"),
    col("TrackId").alias("item"),
    col("UserTrackInteractionCount").alias("rating")
).dropna()

# Ensure 'rating' column is numeric
als_data = als_data.withColumn("rating", col("rating").cast("double"))

# Split data into training and validation sets
(training, validation) = als_data.randomSplit([0.8, 0.2], seed=42)

# Run TrainValidationSplit for hyperparameter tuning
best_model, best_params = tune_als_with_tvs(training)

print(f"Best Model Parameters: {best_params}")

Best Model Parameters: {'rank': 40, 'regParam': 0.05, 'maxIter': 50, 'alpha': 0.1}


In [76]:
#For the best_model(one after hyperparameter tuning)
similar_users, recommended_tracks, rmse, mae = get_recommendations(
    spark,
    joined_df,
    target_user_id=500,
    num_recommendations=10,
    num_similar_users=5,
    model=best_model
)

print(f"Model Evaluation:")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

print("\nTop 5 Similar Users:")
similar_users.show()

print("\nRecommended Tracks:")
recommended_tracks.show()


Model Evaluation:
Root Mean Squared Error (RMSE): 0.4316293594768517
Mean Absolute Error (MAE): 0.2634595836472932

Top 5 Similar Users:
+------------+----------+
|similar_user|similarity|
+------------+----------+
|        2836|0.82893515|
|        3124| 0.8236726|
|        1215|0.82338315|
|        1506| 0.8179041|
|        4598| 0.8163151|
+------------+----------+


Recommended Tracks:



[Stage 38022:======>                                                (1 + 7) / 8]



+-------+--------------------+-----------+
|TrackId|               Title|     Artist|
+-------+--------------------+-----------+
|      2|     Hold On Loosely|.38 Special|
|      7|          Take On Me|       a-ha|
|     12|      Hard As A Rock|      AC/DC|
|     13|  Have a Drink On Me|      AC/DC|
|      0|    Caught Up In You|.38 Special|
|      4|   Art For Arts Sake|       10cc|
|      3|Hold On Loosely  ...|.38 Special|
|      8|       Back In Black|      AC/DC|
|    102|   Turn Up the Radio|  Autograph|
|     24| Shot Down In Flames|      AC/DC|
+-------+--------------------+-----------+




[Stage 38022:=>             (1 + 7) / 8][Stage 38126:=========>    (7 + 1) / 10]



## Generating a detailed report using ChatGPT API

In [77]:
def get_user_name(joined_df, user_id):
    return joined_df.filter(col("CustomerID") == user_id).select("Name").first()[0]

def get_user_stats(joined_df, user_id):
    # Get total number of unique songs and artists listened
    stats = joined_df.filter(col("CustomerID") == user_id).agg(
        countDistinct("TrackId").alias("total_songs"),
        countDistinct("Artist").alias("total_artists"),
        sum("UserTrackInteractionCount").alias("total_listens")
    ).first()
    return stats

def get_top_songs_and_artists(joined_df, user_id, n=5):
    # Get top songs with their interaction counts
    top_songs = joined_df.filter(col("CustomerID") == user_id) \
        .groupBy("TrackId", "Title", "Artist") \
        .agg({"UserTrackInteractionCount": "sum"}) \
        .orderBy(desc("sum(UserTrackInteractionCount)")) \
        .limit(n)
    
    # Get top artists with their interaction counts
    top_artists = joined_df.filter(col("CustomerID") == user_id) \
        .groupBy("Artist") \
        .agg({"UserTrackInteractionCount": "sum"}) \
        .orderBy(desc("sum(UserTrackInteractionCount)")) \
        .limit(n)
    
    return top_songs.collect(), top_artists.collect()

def analyze_user_character_and_moods(top_songs, top_artists, client):
    # Prepare the song and artist information for ChatGPT
    songs_info = [f"{song.Title} by {song.Artist}" for song in top_songs]
    artists_info = [artist.Artist for artist in top_artists]
    
    prompt = f"""
    Based on these top songs:
    {', '.join(songs_info)}
    
    And top artists:
    {', '.join(artists_info)}
    
    Please provide:
    1. Main Character Energy: Give a creative title and one-line description that captures the essence of this music taste
    2. Musical Moods: Analyze and provide the following mood distribution based on the songs:
       - Love ({22}%)
       - Dramatic ({21}%)
       - Sad ({21}%)
       - Chill ({21}%)
       - Upbeat ({12}%)

    Please note: If the musical mood percentage is lesser than 15 percent, then do not include that mood. 
    Only show moods which are having high percentage and recompute percentage so that it makes 100 percent.
    
    Format your response exactly like this:
    Main Character Energy: [title] - [description]
    Musical Moods: [mood distribution analysis - comma separated and having percentage]
    """
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a music analyst providing insights on listener preferences."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content

def generate_report(target_user_id, target_user_name, similar_users, similar_user_names, recommended_tracks, top_songs, top_artists):
    # Initialize the OpenAI client
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    # Get user stats and character analysis
    stats = get_user_stats(joined_df, target_user_id)
    character_and_moods = analyze_user_character_and_moods(top_songs, top_artists, client)
    
    # Prepare the input for the final report
    similar_users_info = [f"{id}: {name}" for id, name in zip(similar_users, similar_user_names)]
    tracks_info = [f"{track.Title} by {track.Artist}" for track in recommended_tracks]
    top_songs_info = [f"{song.Title} by {song.Artist}" for song in top_songs]
    top_artists_info = [artist.Artist for artist in top_artists]
    
    prompt = f"""
    Generate a creative report for a music recommendation system with the following information:
    
    Target User: {target_user_name} (ID: {target_user_id})
    
    Listening Stats:
    Total Songs: {stats.total_songs}
    Total Artists: {stats.total_artists}
    Total Listens: {stats.total_listens}
    
    {character_and_moods}
    
    Top 5 Songs:
    {', '.join(top_songs_info)}
    
    Top 5 Artists:
    {', '.join(top_artists_info)}
    
    Top Similar Users:
    {', '.join(similar_users_info)}
    
    Recommended Tracks:
    {', '.join(tracks_info)}
    
    Please provide a creative and engaging summary that includes:
    1. A list of top 5 songs, 5 artists, 5 similar users and 5 recommended tracks as well as main character energy and moods
    2. Overview of the user's listening habits and stats
    3. Analysis of their music taste based on top songs and artists
    4. Description of their musical personality and mood preferences
    5. List the name of the top match (Your top match is name_of_the_user!)
    6. Explanation of why they match with their top similar user
    7. Personalized recommendations explanation
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a music analyst providing insights on listener preferences."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


In [78]:
target_user_id = 500
# similar_users, recommendations = get_recommendations(spark, joined_df, target_user_id)

# Get names for target user and similar users
target_user_name = get_user_name(joined_df, target_user_id)
similar_user_ids = [row.similar_user for row in similar_users.collect()]
similar_user_names = [get_user_name(joined_df, user_id) for user_id in similar_user_ids]

# Get top songs and artists for the target user
top_songs, top_artists = get_top_songs_and_artists(joined_df, target_user_id)

# Generate the report
report = generate_report(target_user_id, target_user_name, similar_user_ids, similar_user_names, 
                        recommended_tracks.collect(), top_songs, top_artists)

print(report)

### Music Recommendation Report for Francisco Thomas (ID: 500)

#### Main Character Energy:
Francisco Thomas embodies the spirit of a Rebel Rocker with a penchant for rebellious rock anthems that evoke a lively sense of freedom and resistance.

#### Musical Moods:
Dramatic (30%), Sad (30%), Chill (30%), Upbeat (10%)

#### Overview:
Francisco Thomas has listened to a total of 263 songs by 158 artists, accumulating a total of 428 listens.

#### Music Taste Analysis:
Francisco's top songs include classics like "Take On Me" by a-ha and "Paranoid" by Black Sabbath, with top artists such as AC/DC and Aerosmith dominating the list. His preference for these iconic rock tracks showcases a love for high-energy, rebellious tunes.

#### Musical Personality and Mood Preferences:
Francisco's music taste leans towards dramatic and sad moods, balanced by a significant chill vibe. The upbeat tracks in his playlist add a touch of energy, reflecting a dynamic and versatile musical personality.

#### Top 